<a href="https://colab.research.google.com/github/sridamul/TestingLLM/blob/main/Adrastea-7b-hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install transformers SentencePiece accelerate huggingface_hub

In [ ]:
import transformers, torch
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/Adrastea-7b-v1.0")
model = LlamaForCausalLM.from_pretrained(
        "decapoda-research/Adrastea-7b-v1.0",
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type mistral to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.


ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [ ]:
eos_token_id = tokenizer.eos_token_id
pad_token_id = tokenizer.pad_token_id


instruction = "Give me a Jenkins pipeline script"
inputs = tokenizer(
    f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: {instruction}
### Response:""",
    return_tensors="pt",
)
input_ids = inputs["input_ids"].to("cuda")

generation_config = transformers.GenerationConfig(
    do_sample=True,
    temperature=0.1,
    top_p=0.75,
    top_k=80,
    repetition_penalty=1.5,
    max_new_tokens=128,
)

with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
        eos_token_id=eos_token_id,
        pad_token_id=pad_token_id,
    )
output_text = tokenizer.decode(
    generation_output[0].cuda(), skip_special_tokens=True
).strip()
print(output_text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: Give me a Jenkins pipeline script
### Response: 1) Create your own Jenkinsfile in Git repository and commit it to version control system (VCS). This file will contain all of configuration details for building, testing or deploying code on remote server using Jenkins CI/CD toolchain. Here's how you can create one from scratch with basic syntax structure -  2) Start by defining stages which are basically steps involved during continuous integration process like build stage where source codes gets compiled into executable files; test phase verifies if these binaries work as expected without any issues etc., package step builds Docker images containing application dependencies required at runtime while final deployment makes
